In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot = True)

C:\David\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# --------------------------------------fun used for CNN---------------------------------------------
def Weight_Vairable(shape):
    init = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init)
def Bias_Variable(shape):
    init = tf.constant(0.1, shape = shape)
    return tf.Variable(init)

def Conv_2D(x, W):
    return tf.nn.conv2d(x,W, strides=[1,1,1,1], padding='SAME') # strides = [1,x_stride,y_stride,1]

def Pool(x, stride_val = 2, ksize_val = 2,method = 'max'):
    if method == 'max':
        return tf.nn.max_pool(x, ksize=[1,ksize_val,ksize_val,1], strides = [1,stride_val,stride_val,1], padding='SAME')
    elif method == 'mean':
        return tf.nn.avg_pool(x, ksize=[1,ksize_val,ksize_val,1], strides=[1,stride_val,stride_val,1], padding = 'SAME')
# --------------------------------------fun used for CNN---------------------------------------------
# define the common neural net layer
def add_nn_layer(data, n_features, n_units, keep_prob =None ,activation = None):
    W = tf.Variable(tf.truncated_normal([n_features, n_units],stddev=0.1))
    Bias = tf.Variable(tf.ones([1,n_units])/5)
    Wx_plus_b = tf.matmul(data, W) + Bias
    if keep_prob != None:
        Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
    if activation == None:
        return tf.nn.relu(Wx_plus_b)
    else:
        return activation(Wx_plus_b)

def cal_accuracy(x,y):
    predict_prob = sess.run(nn_ly2, feed_dict = {xs:x, keep_prob:1})
    compare = tf.equal(tf.argmax(predict_prob, 1), tf.argmax(y,1))
    result = tf.reduce_mean(tf.cast(compare, tf.float32))
    return sess.run(result, feed_dict = {xs:x, ys:y, keep_prob:1})*100

In [3]:
# start construting the cnn
xs = tf.placeholder(tf.float32, [None, 784])
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

# reshape the input images
x_image = tf.reshape(xs, [-1,28,28,1]) # [n_smaples, x_dim, y_dim, n_chanels]

# hidden layer1
W_ly1 = Weight_Vairable([5,5,1,32]) # shape = [k_size_x, k_size_y, n_chanels, n_kernels]
Bias_ly1 = Bias_Variable([32])
conv_ly1 = tf.nn.relu(Conv_2D(x_image, W_ly1) + Bias_ly1) # 28X28X32 images get
pool_ly1 = Pool(conv_ly1, method='max') # 14X14X32 images get
# hidden layer2
W_ly2 = Weight_Vairable([5,5,32,64])
Bias_ly2 = Bias_Variable([64])
conv_ly2 = tf.nn.relu(Conv_2D(pool_ly1, W_ly2)+Bias_ly2) # 14X14X64 images get
pool_ly2 = Pool(conv_ly2, method='max') # 7X7X64 images get

# build the nn layer1
pool_ly2_flat = tf.reshape(pool_ly2, [-1, 7*7*64])
nn_ly1 = add_nn_layer(pool_ly2_flat, 7*7*64, 1024, keep_prob, None)
# build the output layer2
nn_ly2 = add_nn_layer(nn_ly1, 1024, 10, None, activation=tf.nn.softmax)

# entropy
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(nn_ly2)))
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict = {xs:batch_xs, ys:batch_ys, keep_prob:0.5})
    if step % 100 == 0:
        print("-------------------------------------------------")
        print("Now is at step {}".format(step))
        print("The loss = {}".format(sess.run(cross_entropy, feed_dict = {xs:batch_xs, ys:batch_ys, keep_prob:1})))
        #print("The training accuracy: {:.2f}".format(cal_accuracy(mnist.train.images, mnist.train.labels)))
        print("The testing accuracy : {:.2f}".format(cal_accuracy(mnist.test.images, mnist.test.labels)))

-------------------------------------------------
Now is at step 0
The loss = 572.5155639648438
The testing accuracy : 12.33
-------------------------------------------------
Now is at step 100
The loss = 39.60729217529297
The testing accuracy : 88.38
-------------------------------------------------
Now is at step 200
The loss = 20.35727310180664
The testing accuracy : 92.81
-------------------------------------------------
Now is at step 300
The loss = 23.094093322753906
The testing accuracy : 94.49
-------------------------------------------------
Now is at step 400
The loss = 7.517819404602051
The testing accuracy : 95.35
-------------------------------------------------
Now is at step 500
The loss = 19.28399658203125
The testing accuracy : 95.98
-------------------------------------------------
Now is at step 600
The loss = 14.829869270324707
The testing accuracy : 96.33
-------------------------------------------------
Now is at step 700
The loss = 13.683694839477539
The testing 

In [5]:
# to save the all data
saver = tf.train.Saver()
save_path = saver.save(sess, "./CNN_MNIST_tf/save_net.ckpt")
print("Save Path = ", save_path)

Save Path =  ./CNN_MNIST_tf/save_net.ckpt
